<a href="https://colab.research.google.com/github/paulodowd/EMATM0054_22-23/blob/main/Labsheets/Core/L4_LineFollowing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Labsheet 4: Line Following

In previous labsheets we have:

- Familiarised with the Arduino IDE, Sketch files, uploading to the 3Pi+, and utilising the Serial interface
- Implemented basic operation of the 3Pi+ motors, and encapsulated this within safe and confident function(s), and utilised a `class`.
- Implemented basic read functionality of the 3 central IR Line sensors facing the work surface, and encapsulated this within function(s).
- Encapsulated the code for the line sensors within a `class`, refactoring the code so we can be confident there are no errors and bugs.
- Explored the use of `millis()` to approximate task-scheduling on the 3Pi+.

In this labsheet we will:
- Utilise prior work with the line sensors and motors to implement line following behaviour on the 3Pi+.
- Develop a **bang-bang** controller, using `logic` to control the robot.
- Implement a calibration routine for the ground sensor.  
- Develop a **weighted-measurement** of the line position beneath the robot, and provide this as input to a `proportional controller` to control the robot motors.


If you have prior experience with programming, you may wish to skip the section for the bang-bang controller.  The bang-bang controller is generally inefficient, but provides a good introduction to the concepts of providing feedback control to the 3Pi+.  It is still recommended to read through the bang-bang controller section regardless. 


<hr><br><br><br><br>

# Line Following, in Overview

In this labsheet, we will continue to work with just the 3 central ground sensors, labelled `DN1`, `DN2` and `DN3` on the 3PI+ and in the documentation.  The black vinyl tape provided with your 3Pi+ fits between the sensors `DN1` and `DN3`.  

<p align="center">
<br>
<img src="https://github.com/paulodowd/EMATM0054_22-23/blob/main/Images/3PI_BottomAnnotated.png?raw=true">
<br>
</p>

We can quickly sketch out a plan to produce line following behaviour from the knowledge we have so far.  In the illustration below, there are 3 examples of the robot in different scenarios with respect to the line:

<p align="center">
<br>
<img src="https://github.com/paulodowd/EMATM0054_22-23/blob/main/Images/Webots_LineFeedback.png?raw=true">
<br>
</p>

We can quickly draft the following logic:

- In **`scenario (A)`**, the central sensor is activated by the black line, and so the robot can continue with it's normal direction of travel.  This is the ideal scenario for line following.  Here, the ideal is to keep the black line between the inactivated outside sensors.  Note that, whilst we (humans) can observe the robot _will_ leave the line, presently the robot does not have this information.

- In **`scenario (B)`**, the robot has veered off the line to the **`left`**, causing the right-most sensor to become activated.  To bring the line back under the centre sensor, the appropriate feedback is to **`turn right`**.

- In **`scenario (C)`**, the robot has veered off the line to the **`right`**, causing the left-most sensor to become activated.  To bring the line back under the centre sensor, the appropriate feedback is to **`turn left`**.

In essence, we can write a controller to keep the black line between the left and right sensors.  Note, there are other ways to utilise this sensor to follow the line.  

**Note:** It is possible to produce quite convincing line following that is actually operating in an error condition.  Make sure you take your time to **validate** how your code is producing the observable behavior of the robot.  Remember that you have tools at your disposal to help you when debugging:
- Audible Beeping
- Serial Monitor
- 3 LEDs on the 3Pi+ circuit board.
- Peers to talk to engaged in the same tasks (discussion helps a lot!)



## Exercise 1: Ground Sensors

Feel free to discuss these questions with your peers.  It is a valuable skill to be able to construct **hypotheses** prior to working with the robot:

1. **Hypothesise**: Given your prior experience of the line-sensors, what do we expect the time-values of the 3 ground sensors to be on white and black surfaces?
  - in the above example, we have considered the central sensor to be "active" on the line, but we could also define this in inverted terms.  
  
2. **Hypothesise**: What motion would you expect to observe in the robot motion if the feedback-signal was inverted?
  - is there a case where line following can be achieved with an inverted feedback signal?

3. **Hypothesise**: An exceptional case not caught with the above logic would be if all three ground-sensors were detecting back simultaneously.  This can happen even though the black line is narrower than the gap between the two outer-most sensors.  Under what condition might this occur?

4. **Hypothesise**: What would be appropriate feedback responses for other exceptional (non-defined) cases of the sensor activation?

5. **Hypothesise**: If your robot was to calculate a performance score within `loop()` for line-following (a **metric**) as it operated:
  - what **`proprioceptive`** information could be used?
  - what **`exteroceptive`** information could be used?

<hr><br><br><br><br>

# Bang-Bang Controller

<p align="center">
<img src="https://github.com/paulodowd/EMATM0054_22-23/blob/main/Images/3PI_PhotoShortLine.jpg?raw=true">
</p>

The most intuitive form of controller we can write for line following is a **`bang-bang`** controller.  A bang-bang controller has this name because the control decisions tend to be sudden and dramatic (bang!), moving between discrete states (bang<->bang).  

A consequence of this can be that there is no gradation in behaviour, or the behaviour looks jerky.  Jerky motion can be a problem, because it can cause the wheels to slip on the surface.  We'll discuss this again in a later labsheet on Odometry.  

Usually, a bang-bang controller is not a good controller to use, but it is a good place to start if you have not programmed before.  It is possible to complete the coursework for Assessment 1 only with a Bang-Bang controller - and it is acceptable to do this.  Later, you will investigate using PID to control the motors, and you can achieve much higher performance this way.

If you have not done much programming before, a Bang-Bang controller will help to develop a sense of how `conditional statements` can be used to branch code, to change the flow of your program, and so change the observable robot behaviour.  

A bang-bang controller is built using a series of **`if()`** statements.  We will use the `if()` statement to check if a condition is true, and if so, effect a specific behaviour.  

We can interpret the above section and write some **`pseudo-code`**:

```c
if( left_sensor == active_threshold ) {
  // Turn right.

} else if( right_sensor == active_threshold ) {
  // Turn left

} else if ( center_sensor == active_threshold ) {
  // Move forwards

} else {
  // Are there other conditions?
  // What is an appropriate response?

}
```

The above example would need significant changes to work with your code.  From Labsheet 3, you will have implemented a sensor reading function.  From Labsheet 2, you will have implemented some motor command functions.  



## Exercise 2: Implement a Bang-Bang Controller 

1. Is a bang-bang controller **`open-loop`** or **`closed-loop`** control?  What is the difference?

2. Using your prior experience taking readings from the ground sensors, decide appropriate thresholds for when each sensor can be considered `active` or `inactive`:
  - decide whether large time measurements indicate the presence of a line or not.
  - decide on some time measurement values to use as a threshold. For example, at what value would we say the line was present?
  - which sensor states will be used to build the bang-bang control logic discussed above?  You could use individual sensor states, or a combination of them.  
  - are these threshold values the same for `DN2`, `DN3` and `DN4`?  Remember that you can inspect the values of your sensors using `Serial.print()` and `Serial.println()`.
  - remember that you can use the `Serial Plotter` built into the Arduino IDE to look at multiple sensor values simultaneously.
  - it is recommended to use `#define` statements to set these values at the top of your program.  This way, they will be easy to find and change.
  

3. Implement the discussed bang-bang controller logic within your `loop()`:
  - to begin with, place your robot on the line.  Do not expect your robot to be able to find the line if it is not on the line to start with.
  - use the black vinyl tape to create a short black line on your work surface.
  - it is recommended you start with `slow` or `low` motor velocities.
  - to begin with, do not implement forward motion.  Instead, work only with turning on the spot (rotation).  Does your robot turn the correct way to get back to central on the line?
  - check that your `feedback signal` turns your robot in the appropriate directions with respect to sensor activation.
  - investigate what happens when your line is placed into difficult places with respect to the line - are any of these a problem you need to address?
  - once you are confident rotation is working properly, implement forward motion.
  - **help**: remember your conditional statement can use: 
    - `<`  less than
    - `<=` less than or equal to
    - `==` equal to
    - `>=` greater than or equal to
    - `>`  greater than
    - `!=` not equal to
  - **help**: what is the functional difference between the two code examples immediately below?
  
```c
  // Example 1
  if( ) {

  } 
  if( ) {

  }

  // Example 2
  if( ) {

  } else if( ) {

  } 
```

4. Does your robot conduct `turn` and `move fowards` operations seperately?  
  - Can these be integrated so that the robot does not stop moving forwards?
    - it may be useful to use your bang-bang logic to set the value of left and right `pwm` or `power` variables, which are then used to command the motors once after the logic.
  - How is performance effected with turning and moving forwards combined?
  - Moving slowly might increase the general reliability of the line following behaviour.  As a thought experiment, what other hypothetical **`task requirements`** would make fast forward speed desirable for the robotic system?
  - What is the quickest forward speed you can utilise and still achieve reliable line-following?
  - If you have not done so already, experiment with more challenging line shapes, such as corners and curves.  

5. What information about the line does the robot have when no sensors detect the black surface?
  - When might this circumstance occur?  Consider the line map provided for the line following challenge.
  - What would be an appropriate response in this condition?
  - What other information is available to the robot that might be useful?

6. Write a function to simply confirm if the robot is on a black line.  The function should report a `true` or `false` value when called.  For example, a function named `lineDetected()` (returning true or false).
  - is there a reason to discriminate between which of the sensors is on a black surface, or can it be any of the 3?  Explain your reasoning, adjust the function if necessary.

7. Use the above function to allow your robot to start off the line and drive forwards until it meets the line.  Once it is on the line, it should activate (begin calling) the bang-bang controller.
  - Consider using a `global` variable and an `if()` statement to switch the robot between these two behaviours, or `states` of operation. 

8. Start your robot off the line, and allow it to travel forward to join and follow the line.  Currently, what is the most extreme <a href="https://en.wikipedia.org/wiki/Angle_of_incidence_(optics)">angle of incidence</a> where your controller can still successfully begin line following?
  - if you were to create a results table of different angles when joining the line, how could you quantify the reliability of the controller?



<hr><br><br><br><br>

# Weighted-Measurement

The bang-bang controller has the disadvantage that the combinations of logic can become increasingly large, complex and unwiedly.  Another severe limitation of the bang-bang controller is that it uses `hard-coded` (fixed) parameter values to determine behaviour.  If there is a substantial change in the environment (such as a change to ambient infra-red light, e.g., the sun coming out), these hard-defined values determining logic might become invalid.  

<p align="center">
<img width="75%" src="https://github.com/paulodowd/EMATM0054_22-23/blob/main/Images/3PI_DN_Interference.png?raw=true">
</p>

It is more desirable if the robot can determine an appropriate response **autonomously** without hard-coded (fixed) threshold values.  Weighted-Measurement in an example technique that will allow us to achieve this aim.  In essence, we are going to take the 3 sensor values, and "weigh them up" to decide what they mean (where the line is).  

Our sensor reading process returns numerical values so it is possible to perform some calculations.  The ground sensor reports a range of values of time representative of a continuous scale of reflectivity.  This range of values provides much more information from the state of reality, as opposed to using our fixed threshold values (true/false).  Later, creating a continuous input signal will allow us to produce a continuous output - smoother motor control.

We can consider that if a sensor elements is only half across a black line, it will not report a "perfect" black value:

<p align="center">
<img width="50%" src="https://github.com/paulodowd/EMATM0054_22-23/blob/main/Images/3PI_DN_ReflectanceHalf.png?raw=true">
</p>

Furthermore, the black tape size fits between the two outer sensor elements, `DN2` and `DN4`.  In which case, there is a continuous range of possible conditions where 1 or 2 or 3 sensing elements are either fully covered, partially covered or not covered at all.  

These properties of the sensor and environment interaction can be used to our advantage.  The numerical readings from the 3 sensor elements can be combined to approximate where the line may be under the whole ground sensor.  By analogy, we can think of this as taking the sensor readings and placing them on to a traditional weigh-scale.  The "weigh-scale" then represents a useful transformation of the data: 

- The direction the scale tips indicates the relative position of the line under the robot, or **which way** to turn.
- The magnitude of tip of the scale indicates how far the line has moved from the centre of the robot, or **how much** to turn.
- when we have both `magnitude` and `direction`, we can think of this as a **`vector`**.  

<p align="center">
<img width="50%" src="https://github.com/paulodowd/EMATM0054_22-23/blob/main/Images/3PI_DN_Weighted.png?raw=true">
</p>

We want to "weigh" each of the sensor readings against each other.  We could first calculate the weight of each sensing element with respect to the theoretical maximum range of the sensor measurements.  However, we already know that the range of measurements can vary depending on the environment conditions (e.g., interference from ambient light).  The below illustrates a typical **`sensor response`** of one sensing element:

<p align="center">
<img width="66%" src="https://github.com/paulodowd/EMATM0054_22-23/blob/main/Images/3PI_CalibrationPlot.png?raw=true">
</p>

Alternatively, the weight of each sensor reading can be calculated with respect to the total activation (`summation`) of the ground sensor in that instance.  Because we expect the value of summation to vary between the uses of our ground sensor, it is useful to `normalise` the sensor values relative to the total activation, creating an output between [ 0.0 : 1.0 ].  This consistent output value range will make subsequent calculations or conditional statements more generalisable.   

<p align="center">
<img width="75%" src="https://github.com/paulodowd/EMATM0054_22-23/blob/main/Images/3PI_WeightedMeasurement.png?raw=true">
</p>

With the normalised sensor values, we have the `proportional weight` of each ground sensor element (in proportion against the total activation).  Because the central sensor is between the left and right sensors, we will utilise the central sensor weight to contribute to both the left and right weights in our final weighted-measurement calculation:

<p align="center">
$w_{left} = L_{left} + ( L_{centre}  * 0.5 )$
</p>
<p align="center">
$w_{right} = L_{right} + ( L_{centre}  * 0.5 )$
</p>
<p align="center">
$e_{line} = w_{left} - w_{right}$
</p>



where $L$ are ground sensors. For both $w_{left}$ and $w_{right}$ we add half of the central sensor value, implemented by the `gain` of `0.5` in  both cases.  We can think of this as "sharing the influence", or halfing the influence, between the left and right weights.  For $e_{line}$, we subtract the weighted-right ($w_{right}$) from the weighted-left ($w_{left}$), which provides our `error signal`, $e_{line}$

There are other methods to achieve a weighted line measurement (e.g., <a href="http://www.micromouseonline.com/2011/04/15/simpler-line-follower-sensors/">here</a>), and you are free to determine your own methods.  

We expect the `error signal` to be a value ranging between [ -1.0 : +1.0 ].  We can then use this information to decide which way to turn, and by how much.

We know from previous experience (Labsheet 2) that:
> setting different directions but the same pwm on the motors causes the robot to rotate on the spot.

We can utilise this characteristic of a `two-wheel differential drive mobile robot` to apply our error signal $e_{line}$ directly to our motors, but inversing the sign for one motor.  This will have the effect of driving one wheel forwards and the other backwards, with equal magnitude.

However, $e_{line}$ will be too small to sufficiently drive the motors.  We can ask the question:
> "how can we make $e_{line}$ large enough to operate the motors?"

Simply, we can multiply $e_{line}$ by a constant, which is referred to as a `gain`.  This provides `proprtional control` - the turning power will be in proportion to the error signal by a `gain`.  This gain value needs to scale the response to be appropriate for our task, which is to set a `pwm` value for `analogWrite()`.  We can write this as:

<p align="center">
$u(t) = e_{line}(t) * K_{p}$
</p>

Where:
- $u(t)$ means the control feedback signal provided at time $t$
- $e_{line}(t)$ means the line error signal at time $t$
- $K_{p}$ is our gain for proportional control, irrespective of time $t$
  
This creates a `closed-loop` control system, where the `rotation` of the robot is controlled via feedback from the ground sensor `subsystem`.  













## Exercise 3: Weighted-Line Measurement, Proportional Control

In these exercises, there is a risk that you might mix variable types within your code.  As you write your code, be mindful of what `data type` you variables are in, and what type the calculations are executed in.  In general, avoid compound calculations because they are difficult to debug - it is the same amount of computation to split your calcuations across multiple lines of code.    

You will probably see the following bugs - look out for them!
 - $e_{line}$, $w_{left}$, $w_{right}$, always reporting a value of 0.
  - Did you setup your line sensors properly?
 - $e_{line}$ reporting `NaN` or `inf` - this typically means an error through division by 0.
  - Use an `if()` statement to catch a zero value, and decide what calculation or value should be used instead.
 - etc.

<p>
<br>
</p>

1. **Hypothesise:** When $w_{left}$ and $w_{right}$ are equal, what would we expect the value of $e_{line}$ to be?

2. For your 3PI+ robot, would a negative value of $e_{line}$ provide feedback to turn left or right?  
  - There is not a right answer to this question - you must decide how to implement this within your system.

3.  Implement the weighted-line sensing as discussed above. To start with, ignore motor operation and check that a weighted-measurement can be gained ($e_{line}$).  You may also want to develop this code within `loop()` for now, and then integrate it within a line sensing `class` once you are sure it works reliably.  There are further considerations to make:
  - is your ground sensor `active low` or `active high`?   How could you invert the value of a sensor reading?  Is this necessary?
  - how can you invert the sign of the error signal, $e_{line}$?
  - test that $e_{line}$ meets your expectations by moving your 3Pi+ by hand over different surfaces and the a black line.  Does it have a negative value when you would expect?  
  
4. Begin to add motor operation.  Write code to utilise the weighted-measurement to operate the 3Pi+ motors:
    - **Decompose the problem:** To begin with, ignore any forward motion.
    - You can start by using the Bang-Bang controller code by just checking if the value is positive or negative, and deciding which way to turn.
    - Is your robot able to re-centre itself over the line?  Check by manually placing your robot in different circumstances.  
    - If it turns the wrong way, how might you invert the behaviour?
  - **help**: What would you expect your robot to do when it is not on a line?  What do you observe when the robot is not on a line?
  - **help**: The following code extract should help you to instead utilise the continuous $e_{line}$ value to determine a continous motor output.  You should integrate this with techniques and understanding you have picked up in previous labsheets:

```c
void loop() {

  // Get the line error from a function you have
  // written, or a class you have written.
  float e_line;
  e_line = getLineError(); 

  // Determine a proportional rotation power.  
  // We will scale a maximum PWM value by the 
  // e_line signal.
  float turn_pwm;
  turn_pwm = ????;  // What is a sensible max pwm for rotation?

  // Turn_pwm is scaled by e_line [ -1.0 : +1.0 ]
  turn_pwm = turn_pwm * e_line;

  // Set motor values.
  // Assuming you have written a function to command
  // your motors.
  // What does "0 -" and "0 +" achieve here?
  setMotorValues( (0 - turn_pwm), (0 + turn_pwm) );
    
}

// A function to return an error signal representative
// of the line placement under the ground sensor.
//
// !! Be careful with mixed variable types!!
//
float getLineError() {
  float e_line;
  float w_left;
  float w_right;

  // Get latest line sensor readings.
  // You may need to call an updateLineSensor()
  // function from loop before hand - it depends
  // on your implementation.

  // Sum ground sensor activation

  // Normalise individual sensor readings 
  // against sum

  // Calculated error signal
  w_right = ????;
  w_left  = ????;
  e_line  = ????;

  // Return result
  return e_line;
}

```


5. Modify your solution to implement a fixed forward velocity for your robot in addition to the proportional turning response.
  - How fast can your robot move forwards and still reliably follow the line?  Slowly adjust the speed of your robot, testing each case.
  - Which parts of the line following map present the greatest challenge to your solution?
  - How might your robot measure it's own line following performance (a **`metric`**)?
    - what `proprioceptive` information could be used?
    - what `exteroceptive` information could be used? 
  

6. Implement a mechanism to vary the foward pwm of your robot independently of the turning pwm:
  - Aim for high forward velocity on straight-line segments, and low forward velocity on sharp corners.
  - How can you convert this line performance measure into another `proportional control` mechanism for forward velocity?
  - Is it useful if the forward motion can also be reversed?  e.g., a negative feedback signal?

7. When you are confident that your line following process is good enough, `refactor` the code into a new function with an appropriate name, such as `lineFollowingBehaviour()`.
  - because this behaviour will make operate both the line sensors and the motors, it is not especially useful to integrate this into a class.  When considering your classes, it might be a good criteria to consider what code could be "portable" (useful) to other projects.  
    - In this case, the line following behaviour is likely too specific to your robot - whilst the operation of the motors and line sensors has more generic processes, and more elementary parameters (e.g., pin numbers).  
    - We can also consider portability by how many dependencies a piece of code has on other code.  Ideal portability has the fewest dependencies.

8. Use the task-scheduling technique from Labsheet 3 to schedule the line following behaviour to update at a known time interval:
  - what happens if this time interval is made large?
  - what happens if this time interval is made small?
  - between large and small intervals, is the behaviour consistent in performance? 

9. Alter your code so that your robot can start off-the-line and then drive forwards until it meets the line.  Once it is on the line, it should activate the line following controller.
  - Consider using a `global` variable and an `if()` statement to switch the robot between these two behaviours. 
  - What is the most extreme <a href="https://en.wikipedia.org/wiki/Angle_of_incidence_(optics)">angle of incidence</a> where your controller can still successfully begin line following?
  - How does this compare to the bang-bang controller?

<hr><br><br><br><br>

# Calibration, Improving Performance

<p align="center">
  <img src="https://github.com/paulodowd/EMATM0054_22-23/blob/main/Images/3PI_LineOutput.png?raw=true">
</p>

The above plot is taken from the Arduino IDE built-in plotter, which was receiving information in CSV format from the 3PI+ (how to do this is documented in Labsheet 1).  

The red, blue and green lines show the time measurement for `DN2`, `DN3`, and `DN4` repsectively.  The 3PI+ robot was moved from a white surface to a black surface, which is apparent in the sudden change of values.  Even though the 3PI+ robot was resting flat on the surface, we can observe that the 3 sensors record different measurements for the same surface tone. We can observe this as the seperation between the red, blue and green lines for any point on the x-axis.

If we look very closely at the Arduino plot:
  - the small jagged quality to the lines is `random error`.  
  - the consistent displacement of the line from 0 on the y-axis, and the seperation between the lines, is `systematic error`.

Calibration can be used to help remove or mitigate `systematic error`.  

<p align="center">
<img width="66%" src="https://github.com/paulodowd/EMATM0054_22-23/blob/main/Images/3PI_CalibrationPlot.png?raw=true">
</p>

From the above illustration of a single `sensor repsonse curve`, we can see four characteristics annotated:
- maximum value
- minimum value
- range
- offset bias

A calibration routine can capture these characteristics, and then use them to `condition` the signal.  There are other characteristics we will not consider, such as the slope of the line, or the line linearity.  Calibration makes the assumption that these sources of error are generally consistent (i.e. systematic, expressed of the system).  When systematic error exhibits slow change, calibration may need to be updated at regular intervals.  If your signal is fluctating quickly and unpredictably, you have `random error`, and a better approach is to `filter` the signal.

Because we know that the surface is not changing beyond our expectations, the height of the sensors is not changing, and we can **control** the lighting conditions, we use the robot and it's interaction with the surface to attempt to:
- remove the offset bias, so all sensor signals start at 0
- understand the range, so all sensors report a measurement within an equal range.

A calibration routine is useful because it will permit more meaningful comparison between our 3 sensors, and it may remove bias you were able to see in the output behaviour - for example, your robot may have always drifted to the right in its motion because of the offset bias in the line sensors. 

Our calibration routine will follow a simple `protocol` (series of steps):

1. Initialise the characteristics of the sensor to a known intial value.
2. Activate the sensors for measurement.
3. Activate the actuators to move the robot, such that the sensors will be exposed to the fullest range of conditions expected in the `task environment`.
4. Capture measurements from the sensors over time.  A large number of `samples` is important, but caution must be taken with computer memory and `data types`. 
5. Stop the motion and sensor measurements.
6. Analyse the measurements to determine key characteristics.
7. Utilise the measured characteristics to set conditioning variables.
8. Stop calibration, and apply the conditioning variables to all subsequent readings of the sensor.  





## Exercise 4: Calibration Function 

We will utilise the calibration routine as the first operation when the robot is powered on.  After this, we expect the 3Pi+ into the more complex and iterative behavior of solving the line following challenge.  As such, it is acceptable to write a short sequential section of code which moves through the calibration protocol outlined above.  We can consider calibration as part of the robot initialisation.

Also note that, after your robot has completed this initial calibration, the task environment is likely to change.  This means you may see your sensors exhibit some bias later.  For example, you might find that when centered over the line, you read a small non-zero value despite having completed the calibration.  Here, it is important to ask how accurate and precise you need your sensors to be.  It is likely that you do not need perfect performance.  For example, a small bias (+/-0.1) will be corrected by the closed-loop line following behaviour you will have programmed.  Trying to remove a small error will take a lot of time, and it is likely unncessary to do so.

When you work on this exercise, it is recommended that you save your prior work, and start a new file.  Once you have calibration working, you might consider merging the new calibration code with prior work into a new subsequent file.  This process will help to reduce bugs, and ensure that any progress you have made so far is not lost through confusion.

The following exercises are suggestions, and you are free to explore other calibration techniques:

1. Decide whether:
  - your robot will move continuously for a period of time, and collect samples of both black and white.  E.g., rotating on the spot, over a black and white surface.
  - take two discrete actions, such as measurements recorded on white, and then measurements recorded on black. E.g., beginning on a white surface, then moving forwards and stopping-on a black surface.

2. Have your robot take `n` number of measurements on both black and white surfaces.  Decide whether:
  - your robot will select the minimum and/or maximum from `n` samples.
  - your robot will utilise the mean, median and/or mode from `n` samples. 
  - how this is acheived, or which are appropriate, also depends on how your robot moves (interacts with the environment).

3. Set persistent variables for each of the 3 sensors to store the offset bias.
  - **help**: the offset bias is the consistent value away from 0 on the y-axis.  Is this measured over a black surface or white surface?

4. Utilsing the calculated range (from black to white surfaces), set persistent variables for each of the 3 sensors to store the independent value $(1/range)$.  This will be the `scaling factor` ($S$) to apply to all subsequent readings for each sensor signal.  

5.  Make a call from `setup()` to your calibration function.  Ensure that the robot can perform the calibration as you expect.

6.  Write a new additional function to report a measurement from a sensor which applies the signal conditioning.  Call this function from within `loop()` reporting the values to the Serial Monitor:
  - $c_{i} = (m_{i} - b_{i}) * S_{i}$
  - where:
    - $c_{i}$ is the resulting conditioned measurement for sensor $i$
    - $m_{i}$ is the unconditioned, or raw measurement of sensor $i$
    - $b_{i}$ is the offset bias for sensor $i$
    - $S_{i}$ is the scaling factor for sensor $i$, in range [ 0 : 1 ]

7. Integrate your new calibration routine, and conditioned readings, into either your bang-bang solution or weighted-measurement line following solution.  
  - Instead of working with old code, look to merge your code into a new file. 
  - You should expect that key parameters will need to be adjusted.  
  - Validate the peformance of your robot through observations (`qualitative evaluation`).  What descriptions of motion are you looking for, how would you communicate the criteria you are using?
  - If you developed a method for your 3Pi+ to calculate a measure of performance (a `metric`) for line following previously, now use the same metric in this new system and compare performance (`quantitative evaluation`).  